In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
import torchvision

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from splitDataset import split_large_images

In [ ]:
# Fake dataset should have images and labels folder inside
path = "/media/charles/DATA/Programs/datasetManipulation/Notebooks/fake_dataset3"
if os.path.isdir("/media/charles/DATA/Programs/datasetManipulation/Notebooks/split"):
    os.system('rm -rf /media/charles/DATA/Programs/datasetManipulation/Notebooks/split')
out = split_large_images(path)

In [ ]:
def display_images_with_annotations(images, anns, titles=None, cols=2):
    from datasetUtils import colorFromClass
    from matplotlib import pyplot as plt
    import numpy as np
    import cv2
    
    rows = (len(images) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(12, 12))
    axes = axes.flatten()

    for i, (image, ann, ax) in enumerate(zip(images, anns, axes)):
        if titles is not None:
            ax.set_title(titles[i])

        #img = cv2.imread(image)
        ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert from BGR to RGB for matplotlib

        # Read annotation file and draw polygons
        cls = 0
        for polygon in ann.values():
            polygon *= np.array([image.shape[1], image.shape[0]])
            faceColor, contourColor = colorFromClass(cls)
            ax.fill(*zip(*polygon), facecolor=faceColor, edgecolor=contourColor, alpha=0.7)

        ax.axis('off')

    # Hide empty subplots
    for j in range(len(images), rows * cols):
        axes[j].axis('off')
        axes[j].set_visible(False)

    plt.tight_layout()
    plt.show()

In [ ]:
display_images_with_annotations(out[0], out[1])

In [ ]:
datadir = "/media/charles/DATA/Programs/datasetManipulation/Notebooks/split/images"

def display_imgs_from_folder(datadir):
    from ultralytics.data import YOLODataset
    from ultralytics.utils import LOGGER
    import cv2
    import matplotlib.pyplot as plt
    import numpy as np
    from datasetUtils import colorFromClass
    
    # NOTE: add placeholder to pass class index check
    
    dataset = YOLODataset(datadir, data=dict(names=list(range(1000))))
    
    for l in dataset.labels:
        im = cv2.imread(l["im_file"])
        ax = plt.figure()
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        
        h, w = l["shape"]
        boxes = l["bboxes"]
        if len(boxes) == 0:  # skip empty labels
            continue
        
        boxes[:, [0, 2]] *= w
        boxes[:, [1, 3]] *= h
        
        cls = l["cls"]
        for i,polygon in enumerate(l["segments"]):
            polygon *= np.array([im.shape[1], im.shape[0]])
            faceColor, contourColor = colorFromClass(str(int(cls[i,0])))
            plt.fill(*zip(*polygon), facecolor=faceColor, edgecolor=contourColor, alpha=0.7)

display_imgs_from_folder(datadir)